# Loss Functions and `LossMetrics`

- `x is not implemented for y` bug
- `Flatten` layer
- `CrossEntropyLossFlat`, `activation`, `decodes`

- `LossMetrics` for Style Transfer

In [ ]:
!pip install wwf -q

In [ ]:
from wwf.utils import state_versions

state_versions(['fastai', 'fastcore', 'wwf'])


---
This article is also a Jupyter Notebook available to be run from the top down. There
will be code snippets that you can then run in any environment.

Below are the versions of `fastai`, `fastcore`, and `wwf` currently running at the time of writing this:
* `fastai`: 2.2.7 
* `fastcore`: 1.3.19 
* `wwf`: 0.0.13 
---

In [ ]:
from fastai.vision.all import *

In [ ]:
set_seed(99, True)
path = untar_data(URLs.PETS)/'images'
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2,
    label_func=lambda x: x[0].isupper(), item_tfms=Resize(224))

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [ ]:
x,y = dls.valid.one_batch()

In [ ]:
type(x), type(y)

(fastai.torch_core.TensorImage, fastai.torch_core.TensorCategory)

In [ ]:
with torch.no_grad():
    learn.model.cuda()
    learn.eval()
    out = learn.model(x)

In [ ]:
out[0]

tensor([ 3.8533, -5.2950], device='cuda:0')

In [ ]:
learn.loss_func(out, y)

TensorBase(1.2670, device='cuda:0')

In [ ]:
learn.loss_func

FlattenedLoss of CrossEntropyLoss()

In [ ]:
nn.CrossEntropyLoss()(out, y)

TensorCategory(1.2670, device='cuda:0')

In [ ]:
out.argmax(dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [ ]:
a = TensorMask(1)
b = TensorCategory(1)

In [ ]:
a + b

TypeError: ignored

In [ ]:
def _contiguous(x): return TensorBase(x.transpose(-1,-1))

In [ ]:
def loss(a,b):
    a,b = map(_contiguous, (a,b))

In [ ]:
_contiguous(a)

TensorBase(1)

In [ ]:
a,b = map(_contiguous, (a,b))

In [ ]:
a + b

TensorBase(2)

In [ ]:
a.meta = "This is my metadata"
b.meta = "This is my other metadata"

In [ ]:
(a+b).meta

'This is my metadata'

In [ ]:
(b+a).meta

'This is my other metadata'

In [ ]:
a.transpose(-1,-1).contiguous()

TensorBase(1)

In [ ]:
### Second example

In [ ]:
net = resnet34(num_classes=dls.c)

In [ ]:
with torch.no_grad():
    net.eval()
    net.cuda()
    out_py = net(x)

In [ ]:
type(out_py)

torch.Tensor

In [ ]:
Flatten??

In [ ]:
class LinModel(Module):
    """
    Very basic model that performs y = mx+b. Is used only as a placeholder model for `BaseLearner`.
    Is based on https://github.com/fastai/fastai/blob/master/fastai/test_utils.py#L30
    """
    def __init__(self):
        self.a,self.b = nn.Parameter(torch.randn(1)),nn.Parameter(torch.randn(1))

        self.flatten = Flatten(full=True)

    def forward(self, x): 
        out = x*self.a + self.b
        return self.flatten(out)

In [ ]:
net = LinModel()

In [ ]:
x = TensorImage(1)

In [ ]:
net(x)

TensorBase([-1.0598], grad_fn=<AliasBackward>)

In [ ]:
learn.model

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  